**A SPELLING CORRECTOR**


There are several English words that confuse learners of the language and these words tend to be misspelt when they are written examples are homophones (words that are pronounced the same) and homographs (words written/spelt the same) in English. The goal of this project is to build an automatic spelling corrector to correct misspelt English words with the aid of deep learning and pytorch. 

Transformer is the state of the art deep learning models that produces excellent results for solving NLP sequence to sequence tasks because it learns the context of sentences and monitors the association of linguistic units in sequential data.

For our model, we will be finetuning with a T5 encoder-decoder language model that takes in a text as an input and returns a text as an output. This suits our need. The T5 model has been trained on huge amounts of text-to-text data.

Below shows how the spelling corrector model was built with a T5 model.

# **Install libraries**

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install datasets tqdm pandas

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 462.8/462.8 KB 33.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.3/190.3 KB 26.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.0/132.0 KB 17.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 213.0/213.0 KB 30.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 140.6/140.6 KB 17.9 MB/s eta 0:00:00
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3


In [ ]:
!pip install sentencepiece

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 44.7 MB/s eta 0:00:00


In [ ]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 103.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 100.7 MB/s eta 0:00:00


In [ ]:
!pip install wandb

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 48.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 184.0/184.0 KB 712.8 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 178.9/178.9 KB 21.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 KB 8.8 MB/s eta 0:00:00
  Created wheel for pathtools: filename=pathtools-0.1.2-py3-none-any.whl size=8806 sha256=637f149122bcb3e5b67345574d5d091af8524d8aabbf036f15af62eb229f3dcf
  Stored in directory: /root/.cache/pip/wheels/4c/8e/7e/72fbc243e1aeecae64a96875432e70d4e92f3d2d18123be004
Successfully built pathtools


# **Import Libraries**

In [ ]:
import pandas as pd
from datasets import load_dataset
from tqdm import tqdm

In [ ]:
import argparse
import glob
import os
import json
import time
import logging
import random
import re
from itertools import chain
from string import punctuation

import nltk
nltk.download('punkt')
from nltk.tokenize import sent_tokenize

import pandas as pd
import numpy as np
import torch
from torch.utils.data import Dataset, DataLoader

from transformers import (
    AdamW,
    T5ForConditionalGeneration,
    T5Tokenizer,
    get_linear_schedule_with_warmup
)
 

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


# **Load The Dataset**

I collected from https://github.com/cameronehrlich/homz - a list of commonly confused English words. I modified the dataset by creating a csv file with input and output columns - input is the sentence that is ungrammatical based on the given context with the misseplt English words and output is the correct grammatical structure of the English words as used in context.

In [ ]:
df = pd.read_csv('/content/drive/MyDrive/Spelling/data.csv')
df.shape

(1245, 2)

In [ ]:
df = df.dropna()
df.shape

(1245, 2)

In [ ]:
df.head()

,input,output
0,I have for in my Family Dad Mum and sister .,I have four in my Family Dad Mum and sister .
1,My Dad work's at Melton,My Dad works at Melton
2,My siter go to Tonbury,My sister goes to Tonbury .
3,My Mum goes out some_times.,My Mum goes out sometimes.
4,i go too the Youth clob.,i go to the Youth club .


# **Preprocess the dataset for T5 Model**

Preprocess the data to be in the right format for the model by instantiating the tokenizer of the T5-base model and then return a dictionary containing an input_ids and an attention_mask arrays containing the token ids and the attention masks.

In [ ]:
from transformers import (
    T5ForConditionalGeneration, T5Tokenizer, 
    Seq2SeqTrainingArguments, Seq2SeqTrainer, DataCollatorForSeq2Seq
  )

from torch.utils.data import Dataset, DataLoader
     

In [ ]:
model_name = 't5-base'
tokenizer = T5Tokenizer.from_pretrained(model_name)
model = T5ForConditionalGeneration.from_pretrained(model_name)

/usr/local/lib/python3.8/dist-packages/transformers/models/t5/tokenization_t5.py:163: FutureWarning: This tokenizer was incorrectly instantiated with a model max length of 512 which will be corrected in Transformers v5.
For now, this behavior is kept to avoid breaking backwards compatibility when padding/encoding with `truncation is True`.
- Be aware that you SHOULD NOT rely on t5-base automatically truncating your input to 512 when padding/encoding.
- If you want to encode/pad to sequences longer than 512 you can either instantiate this tokenizer with `model_max_length` or pass `max_length` when encoding/padding.
- To avoid this warning, please instantiate this tokenizer with `model_max_length` set to your preferred value.
  warnings.warn(


In [ ]:
def calculate_token_length(example):
  return len(tokenizer(example).input_ids)

In [ ]:
from sklearn.model_selection import train_test_split
train_df, test_df = train_test_split(df, test_size=0.10, shuffle=True)
train_df.shape, test_df.shape

((1120, 2), (125, 2))

In [ ]:
test_df['input_token_length'] = test_df['input'].apply(calculate_token_length)

In [ ]:
test_df.head()

,input,output,input_token_length
784,I hope this letter will chair you up .,I hope this letter will cheer you up .,11
509,I miss hit,I miss it,4
1162,THE man whore a facemark,THE man wore a facemask,9
483,I hate the food that you served,I ate the food that you served,8
701,The shepherd soon lost site of them in the dar...,The shepherd soon lost sight of them in the da...,12


In [ ]:
test_df['input_token_length'].describe()

count    125.000000
mean      13.120000
std        7.978398
min        4.000000
25%        8.000000
50%       12.000000
75%       16.000000
max       50.000000
Name: input_token_length, dtype: float64

In [ ]:
#spliting the dataset into - test and train
from datasets import Dataset
train_data = Dataset.from_pandas(train_df)
test_data = Dataset.from_pandas(test_df)

In [ ]:
test_data

Dataset({
    features: ['input', 'output', 'input_token_length', '__index_level_0__'],
    num_rows: 125
})

In [ ]:
from torch.utils.data import Dataset, DataLoader
class SpellingDataset(Dataset):
    def __init__(self, dataset, tokenizer,print_text=False):         
        self.dataset = dataset
        self.pad_to_max_length = False
        self.tokenizer = tokenizer
        self.print_text = print_text
        self.max_len = 64
  
    def __len__(self):
        return len(self.dataset)


    def tokenize_data(self, example):
        input_, target_ = example['input'], example['output']

        # tokenize inputs
        tokenized_inputs = tokenizer(input_, pad_to_max_length=self.pad_to_max_length, 
                                            max_length=self.max_len,
                                            return_attention_mask=True)
    
        tokenized_targets = tokenizer(target_, pad_to_max_length=self.pad_to_max_length, 
                                            max_length=self.max_len,
                                            return_attention_mask=True)

        inputs={"input_ids": tokenized_inputs['input_ids'],
            "attention_mask": tokenized_inputs['attention_mask'],
            "labels": tokenized_targets['input_ids']
        }
        
        return inputs

  
    def __getitem__(self, index):
        inputs = self.tokenize_data(self.dataset[index])
        
        if self.print_text:
            for k in inputs.keys():
                print(k, len(inputs[k]))

        return inputs
     

In [ ]:
dataset = SpellingDataset(test_data, tokenizer, True)
print(dataset[12])

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


input_ids 34
attention_mask 34
labels 34
{'input_ids': [27, 1219, 33, 116, 27, 3666, 45, 82, 4537, 7, 16, 8, 7788, 27, 31, 26, 394, 253, 82, 385, 7669, 4354, 42, 255, 228, 1605, 160, 2039, 67, 274, 160, 2053, 5, 1], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], 'labels': [27, 1219, 160, 116, 27, 3666, 45, 82, 4537, 7, 16, 8, 7788, 27, 31, 26, 394, 253, 82, 385, 7669, 4354, 42, 255, 228, 1605, 160, 2039, 67, 274, 160, 2053, 5, 1]}


**Getting Ready to Train Model**

**Evaluation Metrics**

The *rouge_source* is the metric that will be used to evaluate the trainer. 

In [ ]:
!pip install rouge_score

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Preparing metadata (setup.py) ... done
  Created wheel for rouge_score: filename=rouge_score-0.1.2-py3-none-any.whl size=24955 sha256=ab6beaa0dd9c90da09211c20891d959cfbd396393b5c29f0ed7bed1630078ab6
  Stored in directory: /root/.cache/pip/wheels/24/55/6f/ebfc4cb176d1c9665da4e306e1705496206d08215c1acd9dde
Successfully built rouge_score


In [ ]:
from datasets import load_metric
rouge_metric = load_metric("rouge")

<ipython-input-24-76fa4d2ef6b0>:2: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  rouge_metric = load_metric("rouge")


# **Defining Arguments for Training**

The Seq2Seq trainer was instantaiated for the model and the arguments needed for training the model were defined.

In [ ]:
data_collator = DataCollatorForSeq2Seq(tokenizer, model=model, padding='longest', return_tensors='pt')

In [ ]:
batch_size = 16
args = Seq2SeqTrainingArguments(output_dir="/content/drive/MyDrive/Spelling/weights",
                        evaluation_strategy="steps",
                        per_device_train_batch_size=batch_size,
                        per_device_eval_batch_size=batch_size,
                        learning_rate=2e-5,
                        num_train_epochs=1,
                        weight_decay=0.01,
                        save_total_limit=2,
                        predict_with_generate=True,
                        gradient_accumulation_steps = 6,
                        eval_steps = 500,
                        save_steps = 500,
                        load_best_model_at_end=True,
                        logging_dir="/logs",
                        report_to="wandb")

In [ ]:
import nltk
nltk.download('punkt')
import numpy as np

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)
    # Replace -100 in the labels as we can't decode them.
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)
    
    # Rouge expects a newline after each sentence
    decoded_preds = ["\n".join(nltk.sent_tokenize(pred.strip())) for pred in decoded_preds]
    decoded_labels = ["\n".join(nltk.sent_tokenize(label.strip())) for label in decoded_labels]
    
    result = rouge_metric.compute(predictions=decoded_preds, references=decoded_labels, use_stemmer=True)
    # Extract a few results
    result = {key: value.mid.fmeasure * 100 for key, value in result.items()}
    
    # Add mean generated length
    prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in predictions]
    result["gen_len"] = np.mean(prediction_lens)
    return {k: round(v, 4) for k, v in result.items()}
     

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [ ]:
# defining trainer using 🤗
trainer = Seq2SeqTrainer(model=model, 
                args=args, 
                train_dataset= SpellingDataset(train_data, tokenizer),
                eval_dataset=SpellingDataset(test_data, tokenizer),
                tokenizer=tokenizer,
                data_collator=data_collator,
                compute_metrics=compute_metrics)

**Fine-tune The T5 Model**

In [ ]:
trainer.train()

/usr/local/lib/python3.8/dist-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 1120
  Num Epochs = 1
  Instantaneous batch size per device = 16
  Total train batch size (w. parallel, distributed & accumulation) = 96
  Gradient Accumulation steps = 6
  Total optimization steps = 11
  Number of trainable parameters = 222903552
Automatic Weights & Biases logging enabled, to disable set os.environ["WANDB_DISABLED"] = "true"
ERROR:wandb.jupyter:Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.


<IPython.core.display.Javascript object>

wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


Step,Training Loss,Validation Loss




Training completed. Do not forget to share your model on huggingface.co/models =)




TrainOutput(global_step=11, training_loss=1.5620061700994319, metrics={'train_runtime': 31.5312, 'train_samples_per_second': 35.52, 'train_steps_per_second': 0.349, 'total_flos': 50048726630400.0, 'train_loss': 1.5620061700994319, 'epoch': 0.94})

In [ ]:
trainer.save_model('t5_spell_model')

Saving model checkpoint to t5_spell_model
Configuration saved in t5_spell_model/config.json
Configuration saved in t5_spell_model/generation_config.json
Model weights saved in t5_spell_model/pytorch_model.bin
tokenizer config file saved in t5_spell_model/tokenizer_config.json
Special tokens file saved in t5_spell_model/special_tokens_map.json


In [ ]:
!zip -r 't5_spell_model.zip' 't5_spell_model'

  adding: t5_spell_model/ (stored 0%)
  adding: t5_spell_model/special_tokens_map.json (deflated 86%)
  adding: t5_spell_model/tokenizer_config.json (deflated 82%)
  adding: t5_spell_model/pytorch_model.bin (deflated 13%)
  adding: t5_spell_model/spiece.model (deflated 48%)
  adding: t5_spell_model/generation_config.json (deflated 29%)
  adding: t5_spell_model/training_args.bin (deflated 49%)
  adding: t5_spell_model/config.json (deflated 62%)


In [ ]:
!mv t5_spell_model.zip /content/drive/MyDrive/Spelling


**Test The model**

In [ ]:
import torch
from transformers import T5Tokenizer, T5ForConditionalGeneration
model_name = 't5_spell_model'
torch_device = 'cuda' if torch.cuda.is_available() else 'cpu'
tokenizer = T5Tokenizer.from_pretrained(model_name)
model = T5ForConditionalGeneration.from_pretrained(model_name).to(torch_device)

def correct_spelling(input_text,num_return_sequences):
  batch = tokenizer([input_text],truncation=True,padding='max_length',max_length=64, return_tensors="pt").to(torch_device)
  translated = model.generate(**batch,max_length=64,num_beams=4, num_return_sequences=num_return_sequences, temperature=1.5)
  tgt_text = tokenizer.batch_decode(translated, skip_special_tokens=True)
  return tgt_text

loading file spiece.model
loading file added_tokens.json
loading file special_tokens_map.json
loading file tokenizer_config.json
loading configuration file t5_spell_model/config.json
Model config T5Config {
  "_name_or_path": "t5-base",
  "architectures": [
    "T5ForConditionalGeneration"
  ],
  "d_ff": 3072,
  "d_kv": 64,
  "d_model": 768,
  "decoder_start_token_id": 0,
  "dense_act_fn": "relu",
  "dropout_rate": 0.1,
  "eos_token_id": 1,
  "feed_forward_proj": "relu",
  "initializer_factor": 1.0,
  "is_encoder_decoder": true,
  "is_gated_act": false,
  "layer_norm_epsilon": 1e-06,
  "model_type": "t5",
  "n_positions": 512,
  "num_decoder_layers": 12,
  "num_heads": 12,
  "num_layers": 12,
  "output_past": true,
  "pad_token_id": 0,
  "relative_attention_max_distance": 128,
  "relative_attention_num_buckets": 32,
  "task_specific_params": {
    "summarization": {
      "early_stopping": true,
      "length_penalty": 2.0,
      "max_length": 200,
      "min_length": 30,
      "no_rep

In [ ]:
text = 'Their is a bag here.'
print(correct_spelling(text, num_return_sequences=2))
     

Generate config GenerationConfig {
  "decoder_start_token_id": 0,
  "eos_token_id": 1,
  "pad_token_id": 0,
  "transformers_version": "4.26.0"
}



['There is a bag here.', "There's a bag here."]


In [ ]:
text = 'he his my man.'
print(correct_spelling(text, num_return_sequences=2))

Generate config GenerationConfig {
  "decoder_start_token_id": 0,
  "eos_token_id": 1,
  "pad_token_id": 0,
  "transformers_version": "4.26.0"
}



['he his my man.', 'he is my man.']
